# NVFlare low-level APIs

In this chapter, we will explore some of the low-level APIs available in NVFlare to build fully custom components and algorithms.

We'll explore the following topics:

- [Building custom components](../09.1_building_custom_components/building_custom_components.ipynb) - we'll discuss how to implement custom `Controller`s and `Executor`s and instantiate them
- [Sending and receiving tasks and data](../09.2_tasks_and_data_share/tasks_and_data_share.ipynb) - we'll see how to send and receive tasks and data between the `Controller` and `Executor`s
- [Peer-to-peer communincation](../09.3_p2p_communication/p2p_communication.ipynb) - we'll see how to implement peer-to-peer communication between `Executor`s
- [Implementing custom P2P algorithms](../09.4_implementing_p2p_algorithms/) - we'll put together what we learned in the previous sections to implement custom peer-to-peer algorithms. As a guiding example, we will walk through how to implement from scratch the `nvflare.app_opt.p2p` module, which implements peer-to-peer (P2P) distributed optimization algorithms.
